In [1]:
import mydemosaicking_header as mh
from mydensenet import DenseNet 

#read data
train_gt_path = 'E:/data/DIV2K/DIV2K_train_patch/train/gt/'
train_train_path = 'E:/data/DIV2K/DIV2K_train_patch/train/train/'
valid_gt_path = 'E:/data/DIV2K/DIV2K_train_patch/valid/gt/'
valid_valid_path = 'E:/data/DIV2K/DIV2K_train_patch/valid/valid/'
McM_test_path = 'E:/data/McM/McM_test/'
McM_gt_path = 'E:/data/McM/McM_gt/'
McM_test_img = []
McM_gt_img = []
train_gt_patch = []
train_train_patch = []
valid_gt_img = []
valid_valid_img = []
rgb = ['r','g','b']

for i in rgb:
    gt_patch = '_gt_patch_all.h5'
    train_patch = '_train_patch_all.h5'
    gt_filename = i+gt_patch
    train_filename = i+train_patch
    temp = mh.load_h5(train_gt_path, gt_filename, 1)
    train_gt_patch.append(temp)
    temp = mh.load_h5(train_train_path, train_filename, 1)
    train_train_patch.append(temp)
#    for j in range(1,101):
#        valid_img = '_valid_img_'
#        gt_img = '_gt_img_'
#        h5 = '.h5'
#        valid_filename = str(j) + valid_img + i + h5
#        gt_filename = str(j) + gt_img + i + h5
#        temp = mh.load_h5(valid_valid_path, valid_filename, j)
#        valid_valid_img.append(temp)
#        temp = mh.load_h5(valid_gt_path, gt_filename, j)
#        valid_gt_img.append(temp)
    for j in range(1,19):
        test_img = '_test_img_'
        gt_img = '_gt_img_'
        h5 = '.h5'
        test_filename = str(j) + test_img + i + h5
        gt_filename = str(j) + gt_img + i + h5
        temp = mh.load_h5(McM_test_path, test_filename, j)
        McM_test_img.append(temp)
        temp = mh.load_h5(McM_gt_path, gt_filename, j)
        McM_gt_img.append(temp)    

Using TensorFlow backend.


In [4]:
BATCH_SIZE = 64
EPOCHS = 200
w_path = './checkpoints/red/weights-improvement-100-35.90.hdf5'

model = DenseNet(nb_dense_block=5, growth_rate=32, nb_filter=64, reduction=0.0, dropout_rate=0.0, weights_path=None)

adam = mh.Adam(lr=0.000001)
model.compile(adam, loss='mse', metrics=[mh.PSNR, "accuracy"])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data (InputLayer)               (None, None, None, 1 0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 640         data[0][0]                       
__________________________________________________________________________________________________
relu1 (Activation)              (None, None, None, 6 0           conv1[0][0]                      
__________________________________________________________________________________________________
conv2_1_x1 (Conv2D)             (None, None, None, 1 8320        relu1[0][0]                      
__________________________________________________________________________________________________
relu2_1_x1

In [5]:
r_filepath="./checkpoints/red/weights-improvement-{epoch:02d}-{PSNR:.2f}.hdf5"
r_checkpoint = mh.ModelCheckpoint(r_filepath, monitor=mh.PSNR, verbose=1, mode='max')
r_callbacks_list = [r_checkpoint]

g_filepath="./checkpoints/green/weights-improvement-{epoch:02d}-{PSNR:.2f}.hdf5"
g_checkpoint = mh.ModelCheckpoint(g_filepath, monitor=mh.PSNR, verbose=1, mode='max')
g_callbacks_list = [g_checkpoint]

b_filepath="./checkpoints/blue/weights-improvement-{epoch:02d}-{PSNR:.2f}.hdf5"
b_checkpoint = mh.ModelCheckpoint(b_filepath, monitor=mh.PSNR, verbose=1, mode='max')
b_callbacks_list = [b_checkpoint]

model.fit_generator(mh.train_image_gen(train_train_patch, train_gt_patch, 1, BATCH_SIZE), steps_per_epoch=522939 // BATCH_SIZE,  \
					validation_data=mh.valid_image_gen(McM_test_img, McM_gt_img, 1, 18), validation_steps=18, \
					epochs=EPOCHS, workers=8, callbacks=g_callbacks_list)

print("Done training!!!")

print("Saving the final model ...")

#model.save('vdsr_model.h5')  # creates a HDF5 file 
del model  # deletes the existing model

Epoch 1/200
8170/8170 [==============================] - 2353s 288ms/step - loss: 3.3672e-04 - PSNR: 36.9073 - acc: 0.0156 - val_loss: 1.3483e-04 - val_PSNR: 40.0698 - val_acc: 0.0029

Epoch 00001: saving model to ./checkpoints/green/weights-improvement-01-36.91.hdf5
Epoch 2/200
8170/8170 [==============================] - 2320s 284ms/step - loss: 1.9262e-04 - PSNR: 37.3818 - acc: 0.0156 - val_loss: 1.3483e-04 - val_PSNR: 40.0698 - val_acc: 0.0029

Epoch 00002: saving model to ./checkpoints/green/weights-improvement-02-37.38.hdf5
Epoch 3/200
8170/8170 [==============================] - 2318s 284ms/step - loss: 1.9261e-04 - PSNR: 37.3819 - acc: 0.0156 - val_loss: 1.3482e-04 - val_PSNR: 40.0702 - val_acc: 0.0029

Epoch 00003: saving model to ./checkpoints/green/weights-improvement-03-37.38.hdf5
Epoch 4/200
8170/8170 [==============================] - 2320s 284ms/step - loss: 1.9259e-04 - PSNR: 37.3825 - acc: 0.0156 - val_loss: 1.3477e-04 - val_PSNR: 40.0728 - val_acc: 0.0029

Epoch 00004


Epoch 00031: saving model to ./checkpoints/green/weights-improvement-31-37.94.hdf5
Epoch 32/200
8170/8170 [==============================] - 2364s 289ms/step - loss: 1.6924e-04 - PSNR: 37.9425 - acc: 0.0156 - val_loss: 1.3229e-04 - val_PSNR: 40.1728 - val_acc: 0.0029

Epoch 00032: saving model to ./checkpoints/green/weights-improvement-32-37.94.hdf5
Epoch 33/200
8170/8170 [==============================] - 2360s 289ms/step - loss: 1.6906e-04 - PSNR: 37.9473 - acc: 0.0156 - val_loss: 1.3228e-04 - val_PSNR: 40.1733 - val_acc: 0.0029

Epoch 00033: saving model to ./checkpoints/green/weights-improvement-33-37.95.hdf5
Epoch 34/200
8170/8170 [==============================] - 2356s 288ms/step - loss: 1.6889e-04 - PSNR: 37.9518 - acc: 0.0156 - val_loss: 1.3227e-04 - val_PSNR: 40.1740 - val_acc: 0.0029

Epoch 00034: saving model to ./checkpoints/green/weights-improvement-34-37.95.hdf5
Epoch 35/200
8170/8170 [==============================] - 2360s 289ms/step - loss: 1.6872e-04 - PSNR: 37.9561

8170/8170 [==============================] - 2361s 289ms/step - loss: 1.6569e-04 - PSNR: 38.0361 - acc: 0.0156 - val_loss: 1.3135e-04 - val_PSNR: 40.2126 - val_acc: 0.0029

Epoch 00062: saving model to ./checkpoints/green/weights-improvement-62-38.04.hdf5
Epoch 63/200
8170/8170 [==============================] - 2362s 289ms/step - loss: 1.6561e-04 - PSNR: 38.0383 - acc: 0.0156 - val_loss: 1.3130e-04 - val_PSNR: 40.2148 - val_acc: 0.0029

Epoch 00063: saving model to ./checkpoints/green/weights-improvement-63-38.04.hdf5
Epoch 64/200
8170/8170 [==============================] - 2364s 289ms/step - loss: 1.6552e-04 - PSNR: 38.0404 - acc: 0.0156 - val_loss: 1.3124e-04 - val_PSNR: 40.2171 - val_acc: 0.0029

Epoch 00064: saving model to ./checkpoints/green/weights-improvement-64-38.04.hdf5
Epoch 65/200
8170/8170 [==============================] - 2364s 289ms/step - loss: 1.6545e-04 - PSNR: 38.0425 - acc: 0.0156 - val_loss: 1.3119e-04 - val_PSNR: 40.2195 - val_acc: 0.0029

Epoch 00065: saving 


Epoch 00092: saving model to ./checkpoints/green/weights-improvement-92-38.09.hdf5
Epoch 93/200
8170/8170 [==============================] - 2363s 289ms/step - loss: 1.6366e-04 - PSNR: 38.0892 - acc: 0.0156 - val_loss: 1.3020e-04 - val_PSNR: 40.2610 - val_acc: 0.0029

Epoch 00093: saving model to ./checkpoints/green/weights-improvement-93-38.09.hdf5
Epoch 94/200
8170/8170 [==============================] - 2364s 289ms/step - loss: 1.6361e-04 - PSNR: 38.0905 - acc: 0.0156 - val_loss: 1.3018e-04 - val_PSNR: 40.2622 - val_acc: 0.0029

Epoch 00094: saving model to ./checkpoints/green/weights-improvement-94-38.09.hdf5
Epoch 95/200
8170/8170 [==============================] - 2364s 289ms/step - loss: 1.6356e-04 - PSNR: 38.0919 - acc: 0.0156 - val_loss: 1.3015e-04 - val_PSNR: 40.2631 - val_acc: 0.0029

Epoch 00095: saving model to ./checkpoints/green/weights-improvement-95-38.09.hdf5
Epoch 96/200
8170/8170 [==============================] - 2365s 290ms/step - loss: 1.6351e-04 - PSNR: 38.0932

8170/8170 [==============================] - 2379s 291ms/step - loss: 1.6232e-04 - PSNR: 38.1243 - acc: 0.0156 - val_loss: 1.2960e-04 - val_PSNR: 40.2865 - val_acc: 0.0029

Epoch 00123: saving model to ./checkpoints/green/weights-improvement-123-38.12.hdf5
Epoch 124/200
8170/8170 [==============================] - 2380s 291ms/step - loss: 1.6228e-04 - PSNR: 38.1253 - acc: 0.0156 - val_loss: 1.2958e-04 - val_PSNR: 40.2876 - val_acc: 0.0029

Epoch 00124: saving model to ./checkpoints/green/weights-improvement-124-38.13.hdf5
Epoch 125/200
8170/8170 [==============================] - 2378s 291ms/step - loss: 1.6225e-04 - PSNR: 38.1264 - acc: 0.0156 - val_loss: 1.2956e-04 - val_PSNR: 40.2884 - val_acc: 0.0029

Epoch 00125: saving model to ./checkpoints/green/weights-improvement-125-38.13.hdf5
Epoch 126/200
8170/8170 [==============================] - 2379s 291ms/step - loss: 1.6221e-04 - PSNR: 38.1274 - acc: 0.0156 - val_loss: 1.2954e-04 - val_PSNR: 40.2891 - val_acc: 0.0029

Epoch 00126: s

8170/8170 [==============================] - 2397s 293ms/step - loss: 1.6129e-04 - PSNR: 38.1517 - acc: 0.0156 - val_loss: 1.2913e-04 - val_PSNR: 40.3062 - val_acc: 0.0029

Epoch 00153: saving model to ./checkpoints/green/weights-improvement-153-38.15.hdf5
Epoch 154/200
8170/8170 [==============================] - 2397s 293ms/step - loss: 1.6126e-04 - PSNR: 38.1525 - acc: 0.0156 - val_loss: 1.2911e-04 - val_PSNR: 40.3070 - val_acc: 0.0029

Epoch 00154: saving model to ./checkpoints/green/weights-improvement-154-38.15.hdf5
Epoch 155/200
8170/8170 [==============================] - 2399s 294ms/step - loss: 1.6123e-04 - PSNR: 38.1533 - acc: 0.0156 - val_loss: 1.2910e-04 - val_PSNR: 40.3076 - val_acc: 0.0029

Epoch 00155: saving model to ./checkpoints/green/weights-improvement-155-38.15.hdf5
Epoch 156/200
8170/8170 [==============================] - 2398s 293ms/step - loss: 1.6120e-04 - PSNR: 38.1541 - acc: 0.0156 - val_loss: 1.2908e-04 - val_PSNR: 40.3081 - val_acc: 0.0029

Epoch 00156: s

8170/8170 [==============================] - 2350s 288ms/step - loss: 1.6046e-04 - PSNR: 38.1742 - acc: 0.0156 - val_loss: 1.2870e-04 - val_PSNR: 40.3225 - val_acc: 0.0029

Epoch 00183: saving model to ./checkpoints/green/weights-improvement-183-38.17.hdf5
Epoch 184/200
8170/8170 [==============================] - 2357s 289ms/step - loss: 1.6043e-04 - PSNR: 38.1749 - acc: 0.0156 - val_loss: 1.2867e-04 - val_PSNR: 40.3233 - val_acc: 0.0029

Epoch 00184: saving model to ./checkpoints/green/weights-improvement-184-38.17.hdf5
Epoch 185/200
8170/8170 [==============================] - 2348s 287ms/step - loss: 1.6040e-04 - PSNR: 38.1756 - acc: 0.0156 - val_loss: 1.2866e-04 - val_PSNR: 40.3237 - val_acc: 0.0029

Epoch 00185: saving model to ./checkpoints/green/weights-improvement-185-38.18.hdf5
Epoch 186/200
8170/8170 [==============================] - 2346s 287ms/step - loss: 1.6038e-04 - PSNR: 38.1762 - acc: 0.0156 - val_loss: 1.2865e-04 - val_PSNR: 40.3243 - val_acc: 0.0029

Epoch 00186: s